# AI Application

AI Applications are a high-level component, or building block, of Marvin. Like all Marvin components, they are completely standalone: you're free to use them with or without the rest of Marvin.

<div class="admonition abstract">
  <p class="admonition-title">What it does</p>
  <p>
    An autonomous and conversational interface to a stateful, AI-powered application.
  </p>
</div>

In [1]:
from datetime import datetime
from pydantic import BaseModel, Field
from marvin import AIApplication


class ToDo(BaseModel):
    title: str
    description: str
    due_date: datetime = None
    done: bool = False


class ToDoState(BaseModel):
    todos: list[ToDo] = []


class ToDoApp(AIApplication):
    state: ToDoState = Field(default_factory=ToDoState)
    description: str = """
        A simple to-do tracker. Users will give instructions to add, remove, and
        update their to-dos.
        """

<div class="admonition info">
  <p class="admonition-title">How it works</p>
  <p>
    Each AI application maintains an internal <code>state</code> and <code>plan</code> and can use <code>tools</code> to interact with the world.
  </p>
</div>

<div class="admonition tip">
  <p class="admonition-title">When to use</p>
  <p>
    Use an AI Application as the foundation of an autonomous agent (or system of agents) to complete arbitrary tasks.
    <li>a ToDo app, as a simple example</li>
    <li>a Slackbot, that can do anything (<a href="/src/guides/slackbot/">see example</a>)</li>
    <li>a router app that maintains a centralized global state and delegates work to other apps based on inputs (like <a href="https://github.com/microsoft/JARVIS">JARVIS</a>)</li>
  </p>
</div>

## Streaming

AI Applications support streaming LLM outputs to facilitate a more friendly and responsive UX. To enable streaming, provide a `streaming_handler` function to the `AIApplication` class. The handler will be called each time a new token is received and provided a `Message` object that contains all data received from the LLM to that point. It can then perform any side effect (such as printing, logging, or updating a UI), but its return value (if any) is ignored.

In [5]:
streaming_app = AIApplication(
    # pretty-print every partial message as received
    stream_handler=lambda msg: print(msg.content)
)

response = streaming_app("What's 1 + 1?")


The
The sum
The sum of
The sum of 
The sum of 1
The sum of 1 and
The sum of 1 and 
The sum of 1 and 1
The sum of 1 and 1 is
The sum of 1 and 1 is 
The sum of 1 and 1 is 2
The sum of 1 and 1 is 2.
The sum of 1 and 1 is 2.


<div class="admonition tip">
  <p class="admonition-title">Per-token callbacks</p>
  <p>
    The streaming handler is called with a <code>Message</code> object that represents all data received to that point, but the most-recently received tokens are stored in a raw ("delta") form and can be accessed as <code>message.data['streaming_delta']</code>.
  </p>
</div>



## Features

#### 🔨 Easy to Extend
AI Applications accept a `list[Tool]`, where an arbitrary python function can be interpreted as a tool - so you can bring your own tools.

#### 🤖 Stateful
AI applications can consult and maintain their own application state, which they update as they receive inputs from the world and perform actions.

#### 📝 Task Planning
AI Applications can also maintain an internal `AppPlan`, a `list[Task]` that represent the status of the application's current plan. Like the application's state, the plan is updated as the application instance evolves.